In [ ]:
import numpy as np
import random

# --- 1. AYNI VERİ ÜRETİMİ (Süreklilik İçin) ---
NUM_COMPONENTS = 400   
NUM_ECUS = 10          
random.seed(42) 

COMPONENTS = [f'C_{i}' for i in range(1, NUM_COMPONENTS + 1)]
ECUS = [f'E_{j}' for j in range(1, NUM_ECUS + 1)]

# Basitleştirilmiş Veri Seti (Anlaşılması kolay olsun diye)
# ECU Kapasiteleri (Puan bazlı düşünelim: 100 puanlık yer var)
ECU_CAPACITY = {e: 100 for e in ECUS}
# ECU Özellikleri (Bazılarında GPU var)
ECU_FEATURES = {e: set() for e in ECUS}
for e in ['E_3', 'E_9', 'E_10']: ECU_FEATURES[e].add('GPU') # HPC'ler
for e in ['E_2', 'E_5', 'E_6', 'E_8']: ECU_FEATURES[e].add('HSM') # Safety

# Bileşen İhtiyaçları
REQ_LOAD = {}
REQ_FEAT = {}
for c in COMPONENTS:
    REQ_LOAD[c] = random.randint(5, 25) # Her biri 5-25 arası yer kaplıyor
    REQ_FEAT[c] = set()
    if random.random() < 0.3: REQ_FEAT[c].add('GPU') # %30'u GPU istiyor
    elif random.random() < 0.3: REQ_FEAT[c].add('HSM')

# Kritik Çiftler (Birbirinden kaçması gerekenler)
CRITICAL_PAIRS = [('C_1', 'C_2'), ('C_5', 'C_6'), ('C_10', 'C_11')]

# Ajanların Zeka Seviyelerini Ata
# %70 L0 (Saf), %30 L1 (Zeki)
LEVELS = {c: 0 if random.random() < 0.7 else 1 for c in COMPONENTS}
# Kritik çiftlerin en az biri L1 olsun ki kaçabilsin
for c1, c2 in CRITICAL_PAIRS:
    LEVELS[c2] = 1 

print(f"Simulation Setup: {len(COMPONENTS)} Components, {len(ECUS)} ECUs.")
print(f"L0 Agents (Probabilistic): {list(LEVELS.values()).count(0)}")
print(f"L1 Agents (Strategic): {list(LEVELS.values()).count(1)}")

# ==============================================================================
# LEVEL-K ALGORİTMASI
# ==============================================================================

def calculate_preference_score(comp, ecu, current_congestion=0):
    """
    Bir bileşenin bir ECU'yu ne kadar istediğini hesaplar (Utility Function).
    """
    score = 10.0 # Baz puan
    
    # 1. Donanım Uyumu (En önemli)
    req = REQ_FEAT[comp]
    avail = ECU_FEATURES[ecu]
    if not req.issubset(avail):
        return 0.0 # Uymuyorsa tercih etme (veya çok düşük puan)
    
    # 2. Kapasite / Sıkışıklık Cezası
    # L0 için current_congestion 0'dır (Görmez). 
    # L1 için doluluk arttıkça puan düşer.
    if current_congestion + REQ_LOAD[comp] > ECU_CAPACITY[ecu]:
        return 0.1 # Taşacaksa gitme
    
    # Sıkışıklık ne kadar azsa puan o kadar yüksek (Load Balancing)
    score += (100 - current_congestion) * 0.5
    
    return score

# --- ADIM 1: L0 TAHMİNİ (PROBABILISTIC BELIEF) ---
# L0 ajanları sadece donanıma bakar, diğerlerinin ne yaptığını bilmez.
# Her ECU için bir "Talep Olasılığı" oluştururlar.

predicted_load_map = {e: 0.0 for e in ECUS} # Beklenen Doluluk
l0_decisions = {} # L0'ların kesin kararı (Simülasyon gereği)

print("\n--- Phase 1: L0 Agents Reasoning ---")

for c in COMPONENTS:
    if LEVELS[c] == 0: # Sadece L0'lar
        scores = {}
        total_score = 0.0
        
        # Her ECU'ya puan ver
        for e in ECUS:
            s = calculate_preference_score(c, e, current_congestion=0) # Congestion bilmiyor
            scores[e] = s
            total_score += s
        
        # Puanları Olasılığa Çevir (HATA BURADAYDI, DUZELTİLDİ)
        probs = {}
        if total_score == 0:
            # Hiçbir yer uymuyor, eşit ihtimalle dağıt (Fallback)
            probs = {e: 1.0/len(ECUS) for e in ECUS}
        else:
            # Düzeltme: score değişkenini döngüden alıyoruz
            probs = {e: score/total_score for e, score in scores.items()}
        
        # 1. Bu olasılıkları "Global Harita"ya işle (L1'ler bunu görecek)
        for e, p in probs.items():
            # Expected Load = Probability * Component_Load
            # e artık düzgün bir string anahtar (örn: 'E_1')
            predicted_load_map[e] += p * REQ_LOAD[c]
            
        # 2. Kendisi için bir seçim yap (Rulet tekerleği seçimi)
        chosen_ecu = random.choices(list(probs.keys()), weights=list(probs.values()))[0]
        l0_decisions[c] = chosen_ecu

print("L0'lar kararlarını verdi ve niyetlerini belli etti.")
print("Tahmini ECU Dolulukları (L1'lerin gördüğü manzara):")
for e in sorted(ECUS):
    print(f"  {e}: %{predicted_load_map[e]:.1f} (Capacity: 100)")


# --- ADIM 2: L1 TAHMİNİ (STRATEGIC RESPONSE) ---
# L1 ajanları, L0'ların yarattığı tahmini yükü görür ve ona göre boş yer arar.
# Ayrıca Isolation kuralına bakar.

final_assignments = l0_decisions.copy() # L0 kararları cepte
l1_decisions = {}

print("\n--- Phase 2: L1 Agents Strategic Move ---")

for c in COMPONENTS:
    if LEVELS[c] == 1: # Sadece L1'ler
        best_ecu = None
        best_val = -1
        
        # Critical Pair Kontrolü
        partner = None
        partner_ecu = None
        for p1, p2 in CRITICAL_PAIRS:
            if c == p1: partner = p2
            elif c == p2: partner = p1
        
        if partner and partner in final_assignments:
            partner_ecu = final_assignments[partner]
        
        # Stratejik Seçim
        for e in ECUS:
            # L1, L0'ların yarattığı tahmini yükü (predicted_load_map) GÖRÜR.
            current_cong = predicted_load_map[e]
            
            val = calculate_preference_score(c, e, current_congestion=current_cong)
            
            # Strateji: Partnerim oradaysa oranın puanını öldür (Safety First)
            if partner_ecu and e == partner_ecu:
                val = -1000 # ASLA GİTME
            
            if val > best_val:
                best_val = val
                best_ecu = e
        
        # Kararı Kaydet
        if best_ecu:
            final_assignments[c] = best_ecu
            # Kendi yükünü haritaya ekle (Sonraki L1'ler görsün diye - opsiyonel)
            predicted_load_map[best_ecu] += REQ_LOAD[c]
        else:
            print(f"WARNING: {c} (L1) could not find a suitable ECU!")

# ==============================================================================
# SONUÇ ANALİZİ
# ==============================================================================
print("\n" + "="*40)
print("       GAME RESULT")
print("="*40)

# 1. Yük Durumu
real_load = {e: 0 for e in ECUS}
for c, e in final_assignments.items():
    real_load[e] += REQ_LOAD[c]

print("Final ECU Loads:")
for e in sorted(ECUS):
    status = "OK" if real_load[e] <= 100 else "OVERLOAD!"
    print(f"  {e}: {real_load[e]}/100 -> {status}")

# 2. Safety (Isolation) Kontrolü
print("\nSafety Isolation Check:")
safe_count = 0
for c1, c2 in CRITICAL_PAIRS:
    e1 = final_assignments.get(c1, "None")
    e2 = final_assignments.get(c2, "None")
    if e1 != e2:
        print(f"  [PASS] {c1}({e1}) <-> {c2}({e2})")
        safe_count += 1
    else:
        print(f"  [FAIL] {c1} and {c2} are both on {e1}!")

print(f"\nSuccess Rate: {safe_count}/{len(CRITICAL_PAIRS)} critical pairs separated.")

Simulation Setup: 40 Components, 10 ECUs.
L0 Agents (Probabilistic): 25
L1 Agents (Strategic): 15

--- Phase 1: L0 Agents Reasoning ---
L0'lar kararlarını verdi ve niyetlerini belli etti.
Tahmini ECU Dolulukları (L1'lerin gördüğü manzara):
  E_1: %20.7 (Capacity: 100)
  E_10: %71.0 (Capacity: 100)
  E_2: %28.7 (Capacity: 100)
  E_3: %71.0 (Capacity: 100)
  E_4: %20.7 (Capacity: 100)
  E_5: %28.7 (Capacity: 100)
  E_6: %28.7 (Capacity: 100)
  E_7: %20.7 (Capacity: 100)
  E_8: %28.7 (Capacity: 100)
  E_9: %71.0 (Capacity: 100)

--- Phase 2: L1 Agents Strategic Move ---

       GAME RESULT
Final ECU Loads:
  E_1: 44/100 -> OK
  E_10: 104/100 -> OVERLOAD!
  E_2: 46/100 -> OK
  E_3: 160/100 -> OVERLOAD!
  E_4: 20/100 -> OK
  E_5: 69/100 -> OK
  E_6: 65/100 -> OK
  E_7: 20/100 -> OK
  E_8: 38/100 -> OK
  E_9: 65/100 -> OK

Safety Isolation Check:
  [PASS] C_1(E_10) <-> C_2(E_3)
  [PASS] C_5(E_3) <-> C_6(E_10)
  [PASS] C_10(E_2) <-> C_11(E_5)

Success Rate: 3/3 critical pairs separated.


In [5]:
import random
import numpy as np

# ==========================================
# 1. AYNI VERİ ÜRETİMİ (Standart)
# ==========================================
NUM_COMPONENTS = 40   
NUM_ECUS = 10          
MAX_LEVELS = 6  # L0, L1, L2 (Döngü derinliği)
random.seed(42) 

COMPONENTS = [f'C_{i}' for i in range(1, NUM_COMPONENTS + 1)]
ECUS = [f'E_{j}' for j in range(1, NUM_ECUS + 1)]

# ECU Kapasiteleri (Puan bazlı: 100 birim)
ECU_CAPACITY = {e: 100 for e in ECUS}
# ECU Özellikleri
ECU_FEATURES = {e: set() for e in ECUS}
for e in ['E_3', 'E_9', 'E_10']: ECU_FEATURES[e].add('GPU')
for e in ['E_2', 'E_5', 'E_6', 'E_8']: ECU_FEATURES[e].add('HSM')

# Bileşen İhtiyaçları
REQ_LOAD = {}
REQ_FEAT = {}
for c in COMPONENTS:
    REQ_LOAD[c] = random.randint(5, 20)
    REQ_FEAT[c] = set()
    r = random.random()
    if r < 0.2: REQ_FEAT[c].add('GPU')
    elif r < 0.5: REQ_FEAT[c].add('HSM')

# Kritik Çiftler
CRITICAL_PAIRS = [('C_1', 'C_2'), ('C_5', 'C_6'), ('C_15', 'C_16')]

# --- LEVEL ATAMA (HIYERARŞİ) ---
# Bileşenleri rastgele seviyelere atıyoruz. 
# Kritik bileşenleri daha yüksek seviyelere (daha zeki) koymak mantıklıdır.
COMPONENT_LEVELS = {}
for c in COMPONENTS:
    COMPONENT_LEVELS[c] = random.choice(range(MAX_LEVELS))

# Safety Hack: Kritik çiftlerin ikinci eşini her zaman bir üst seviyeye taşı
# Böylece ilki yerleştikten sonra, ikincisi ona bakıp kaçabilir.
for c1, c2 in CRITICAL_PAIRS:
    lvl1 = COMPONENT_LEVELS[c1]
    COMPONENT_LEVELS[c2] = min(lvl1 + 1, MAX_LEVELS - 1)

print(f"Simulation Setup: {MAX_LEVELS} Cognitive Levels.")
for k in range(MAX_LEVELS):
    count = list(COMPONENT_LEVELS.values()).count(k)
    print(f"  Level-{k} Agents: {count}")

# ==========================================
# OYUN MOTORU
# ==========================================

# Global State: Şu anki ECU Doluluk Durumu
# Başlangıçta hepsi boş (0)
global_load_map = {e: 0.0 for e in ECUS} 
final_assignments = {}

def calculate_score(comp, ecu, current_load_map):
    """
    Bir bileşenin bir ECU'ya verdiği puan.
    """
    score = 10.0
    
    # 1. Donanım Uyumu (Olmazsa olmaz)
    if not REQ_FEAT[comp].issubset(ECU_FEATURES[ecu]):
        return -1000.0 # Uymuyorsa ceza
    
    # 2. Kapasite Kontrolü
    # current_load_map: Şu ana kadar yerleşenlerin (önceki seviyelerin) yükü
    my_load = REQ_LOAD[comp]
    if current_load_map[ecu] + my_load > ECU_CAPACITY[ecu]:
        return -500.0 # Taşacaksa gitme
    
    # 3. Yük Dengeleme (Boş yeri tercih et)
    score += (100 - current_load_map[ecu]) * 0.2
    
    return score

# ==========================================
# KADEMELİ ÇÖZÜM DÖNGÜSÜ
# ==========================================

print("\n--- STARTING CASCADING GAME ---")

for k in range(MAX_LEVELS):
    print(f"\n>>> Level-{k} Phase Initiated...")
    
    # Bu seviyedeki ajanları (bileşenleri) bul
    current_agents = [c for c, lvl in COMPONENT_LEVELS.items() if lvl == k]
    
    # Bu seviyenin geçici kararları (Aynı seviyedekiler birbirini görmez - Simultaneous Move)
    # Eğer birbirlerini görsünler istersen (Sequential Move), global_load_map'i anlık güncelleriz.
    # Biz "Simultaneous" yapalım, daha kaotik ve gerçekçi olsun.
    level_decisions = {}
    
    for agent in current_agents:
        
        # --- L0 ÖZEL DURUMU (Probabilistic) ---
        if k == 0:
            # L0 kesin konuşmaz, niyet belirtir (Heatmap mantığı)
            # Ama basitlik olsun diye L0'ı da "Greedy" (Açgözlü) yapıp haritaya işleyeceğiz.
            # L0 sadece donanıma bakar, mevcut yüke bakmaz (veya az bakar).
            scores = {e: calculate_score(agent, e, {e:0 for e in ECUS}) for e in ECUS} # Yükü 0 sanıyor
        
        # --- L1, L2... (Strategic) ---
        else:
            # Üst seviyeler, alt seviyelerin oluşturduğu "global_load_map"i görür.
            # Ayrıca Safety kontrolü yaparlar.
            scores = {}
            for e in ECUS:
                base_score = calculate_score(agent, e, global_load_map)
                
                # SAFETY CHECK (CRITICAL PAIRS)
                # Partnerim daha önceki bir seviyede yerleştiyse, ondan kaç!
                partner = None
                for p1, p2 in CRITICAL_PAIRS:
                    if agent == p1: partner = p2
                    elif agent == p2: partner = p1
                
                if partner and partner in final_assignments:
                    if final_assignments[partner] == e:
                        base_score -= 10000.0 # ASLA AYNI YERE GİTME
                
                scores[e] = base_score

        # Karar Anı: En yüksek puanlı ECU'yu seç
        # (Eşitlik durumunda rastgele seç ki yığılma olmasın)
        best_ecus = [e for e, s in scores.items() if s == max(scores.values())]
        chosen_ecu = random.choice(best_ecus)
        
        # Kararı kaydet
        level_decisions[agent] = chosen_ecu
    
    # Faz (Level) Sonu: Kararları Global Haritaya İşle
    # Bir sonraki seviye (L_k+1), bu yükleri "dolu" olarak görecek.
    for agent, ecu in level_decisions.items():
        if calculate_score(agent, ecu, global_load_map) > -100: # Geçerli bir atama ise
            final_assignments[agent] = ecu
            global_load_map[ecu] += REQ_LOAD[agent]
            print(f"  Agent {agent} -> {ecu} (New Load: {global_load_map[ecu]}%)")
        else:
            print(f"  Agent {agent} -> FAILED to find valid ECU (Dropped)")

# ==========================================
# SONUÇ RAPORU
# ==========================================
print("\n" + "="*40)
print("       FINAL CONFIGURATION")
print("="*40)

print("ECU Status:")
for e in sorted(ECUS):
    load = global_load_map[e]
    bar = "|" * int(load/10)
    print(f"  {e}: {load}/100 [{bar:<10}]")

print("\nSafety Verification:")
safe_count = 0
for c1, c2 in CRITICAL_PAIRS:
    e1 = final_assignments.get(c1, "None")
    e2 = final_assignments.get(c2, "None")
    
    status = "SAFE"
    if e1 == e2 or e1 == "None" or e2 == "None":
        status = "VIOLATION"
    else:
        safe_count += 1
    print(f"  Pair {c1}-{c2}: {e1} vs {e2} -> {status}")

unassigned = len(COMPONENTS) - len(final_assignments)
if unassigned > 0:
    print(f"\nWARNING: {unassigned} components could not be assigned due to capacity constraints.")
else:
    print("\nSUCCESS: All components assigned.")

Simulation Setup: 6 Cognitive Levels.
  Level-0 Agents: 5
  Level-1 Agents: 6
  Level-2 Agents: 4
  Level-3 Agents: 7
  Level-4 Agents: 10
  Level-5 Agents: 8

--- STARTING CASCADING GAME ---

>>> Level-0 Phase Initiated...
  Agent C_21 -> E_2 (New Load: 16.0%)
  Agent C_22 -> E_10 (New Load: 7.0%)
  Agent C_23 -> E_9 (New Load: 12.0%)
  Agent C_30 -> E_5 (New Load: 7.0%)
  Agent C_38 -> E_6 (New Load: 11.0%)

>>> Level-1 Phase Initiated...
  Agent C_3 -> E_1 (New Load: 9.0%)
  Agent C_5 -> E_3 (New Load: 6.0%)
  Agent C_17 -> E_7 (New Load: 6.0%)
  Agent C_18 -> E_3 (New Load: 14.0%)
  Agent C_24 -> E_7 (New Load: 13.0%)
  Agent C_26 -> E_1 (New Load: 28.0%)

>>> Level-2 Phase Initiated...
  Agent C_4 -> E_8 (New Load: 7.0%)
  Agent C_6 -> E_8 (New Load: 18.0%)
  Agent C_10 -> E_4 (New Load: 5.0%)
  Agent C_36 -> E_8 (New Load: 24.0%)

>>> Level-3 Phase Initiated...
  Agent C_13 -> E_4 (New Load: 16.0%)
  Agent C_15 -> E_10 (New Load: 24.0%)
  Agent C_20 -> E_4 (New Load: 30.0%)
  Age

In [12]:
import random
import numpy as np

# ==========================================
# 1. VERİ ÜRETİMİ (Standart)
# ==========================================
NUM_COMPONENTS = 4000   
NUM_ECUS = 1000          
MAX_LEVELS = 100 
random.seed(42) 

COMPONENTS = [f'C_{i}' for i in range(1, NUM_COMPONENTS + 1)]
ECUS = [f'E_{j}' for j in range(1, NUM_ECUS + 1)]

# ECU Maliyetleri (HPC pahalı, Gateway ucuz)
ECU_BASE_COST = {}
ECU_CAPACITY = {}
ECU_FEATURES = {e: set() for e in ECUS}

for idx, e in enumerate(ECUS):
    if idx < 3: # HPC
        ECU_BASE_COST[e] = 200
        ECU_CAPACITY[e] = 200 # Daha geniş kapasite
        ECU_FEATURES[e].add('GPU')
    elif idx < 7: # SAFETY
        ECU_BASE_COST[e] = 80
        ECU_CAPACITY[e] = 50
        ECU_FEATURES[e].add('HSM')
    else: # GATEWAY
        ECU_BASE_COST[e] = 30
        ECU_CAPACITY[e] = 30

# Bileşen İhtiyaçları
REQ_LOAD = {}
REQ_FEAT = {}
for c in COMPONENTS:
    REQ_LOAD[c] = random.randint(5, 20)
    REQ_FEAT[c] = set()
    r = random.random()
    if r < 0.2: REQ_FEAT[c].add('GPU')
    elif r < 0.5: REQ_FEAT[c].add('HSM')

# Kritik Çiftler
CRITICAL_PAIRS = [('C_1', 'C_2'), ('C_5', 'C_6'), ('C_15', 'C_16')]

# Level Atama
COMPONENT_LEVELS = {c: random.choice(range(MAX_LEVELS)) for c in COMPONENTS}
# Kritiklerin ikinci eşini üst seviyeye taşı (Safety Reaction için)
for c1, c2 in CRITICAL_PAIRS:
    COMPONENT_LEVELS[c2] = min(COMPONENT_LEVELS[c1] + 1, MAX_LEVELS - 1)

# ==========================================
# MALİYET ODAKLI OYUN MOTORU
# ==========================================

global_load_map = {e: 0.0 for e in ECUS} 
final_assignments = {}

def calculate_cost_aware_score(comp, ecu, current_load_map):
    """
    AMAC: Var olan ECU'ları doldur, yenisini açmaktan kaçın.
    """
    score = 0.0
    
    # 1. Donanım Uyumu (Zorunlu)
    if not REQ_FEAT[comp].issubset(ECU_FEATURES[ecu]):
        return -float('inf') # Asla seçme
    
    # 2. Kapasite Kontrolü
    current_usage = current_load_map[ecu]
    my_load = REQ_LOAD[comp]
    
    if current_usage + my_load > ECU_CAPACITY[ecu]:
        return -float('inf') # Sığmıyor
    
    # 3. COST EFFICIENCY LOGIC (Bin Packing)
    
    # A) ECU Zaten Açık mı?
    is_active = current_usage > 0
    
    if is_active:
        # Zaten açıksa: Oraya gitmek iyidir (Bonus)
        # Ne kadar doluysa o kadar iyi (Boşluk bırakma) -> Density Bonus
        fill_ratio = (current_usage + my_load) / ECU_CAPACITY[ecu]
        score += 500 + (fill_ratio * 100) 
    else:
        # Kapalıysa: Yeni ECU açmak PAHALIDIR (Ceza)
        # Ceza, ECU'nun dolar maliyetiyle orantılı olsun (Pahalı ECU'yu açmak daha kötü)
        score -= ECU_BASE_COST[ecu] * 10 
        
    return score

# ==========================================
# OYUN DÖNGÜSÜ
# ==========================================

print(f"\n--- STARTING COST-EFFICIENT GAME ({len(COMPONENTS)} Components) ---")

for k in range(MAX_LEVELS):
    print(f"\n>>> Level-{k} Phase...")
    current_agents = [c for c, lvl in COMPONENT_LEVELS.items() if lvl == k]
    
    # Daha iyi "Packing" için büyük parçaları önce yerleştirmek mantıklıdır (Heuristic)
    # Bu yüzden ajanları yüklerine göre sıralıyoruz (Büyükten küçüğe)
    current_agents.sort(key=lambda x: REQ_LOAD[x], reverse=True)
    
    level_decisions = {}
    
    for agent in current_agents:
        scores = {}
        for e in ECUS:
            # Puanı hesapla
            base_score = calculate_cost_aware_score(agent, e, global_load_map)
            
            # Safety Check (L1 ve üzeri için)
            if k > 0:
                partner = None
                for p1, p2 in CRITICAL_PAIRS:
                    if agent == p1: partner = p2
                    elif agent == p2: partner = p1
                
                if partner and partner in final_assignments:
                    if final_assignments[partner] == e:
                        base_score = -float('inf') # Safety İhlali
            
            scores[e] = base_score
        
        # En iyi skoru bul
        max_s = max(scores.values())
        if max_s == -float('inf'):
            # Geçerli yer yok
            chosen_ecu = None
        else:
            # En iyilerden birini seç
            best_candidates = [e for e, s in scores.items() if s == max_s]
            chosen_ecu = best_candidates[0] # İlk bulduğunu al (Deterministic packing)
            
        level_decisions[agent] = chosen_ecu
        
        # SEQUENTIAL MOVE HACK:
        # Bin Packing'de "Simultaneous" yerine "Sequential" (Sıralı) karar daha verimlidir.
        # Bir ajan karar verince, haritayı hemen güncelleyelim ki sonraki ajan "Aaa orası açıldı ben de geleyim" desin.
        if chosen_ecu:
            final_assignments[agent] = chosen_ecu
            global_load_map[chosen_ecu] += REQ_LOAD[agent]

# ==========================================
# MALİYET ANALİZİ
# ==========================================
print("\n" + "="*40)
print("       COST EFFICIENCY REPORT")
print("="*40)

total_system_cost = 0
active_ecus = 0

print(f"{'ECU':<5} | {'Type':<8} | {'Load':<10} | {'Cost':<5} | Status")
print("-" * 50)

for e in sorted(ECUS):
    load = global_load_map[e]
    cap = ECU_CAPACITY[e]
    cost = 0
    status = "OFF (0$)"
    
    if load > 0:
        cost = ECU_BASE_COST[e]
        total_system_cost += cost
        active_ecus += 1
        status = "ON"
    
    pct = (load/cap)*100
    print(f"{e:<5} | {'HPC' if e in ['E_1','E_2','E_3'] else 'OTHER':<8} | {int(load)}/{cap} ({int(pct)}%) | {cost:<5} | {status}")

print(f"\nTotal Components: {len(COMPONENTS)}")
print(f"Assigned: {len(final_assignments)}")
print(f"Unassigned: {len(COMPONENTS) - len(final_assignments)}")
print("-" * 30)
print(f"Active ECUs: {active_ecus} / {NUM_ECUS}")
print(f"TOTAL SYSTEM COST: ${total_system_cost}")

# Safety Check
print("\nSafety Check:")
safe = True
for c1, c2 in CRITICAL_PAIRS:
    e1, e2 = final_assignments.get(c1), final_assignments.get(c2)
    if e1 and e2 and e1 == e2:
        print(f"  VIOLATION: {c1} & {c2} on {e1}")
        safe = False
if safe: print("  All Critical Pairs Separated.")


--- STARTING COST-EFFICIENT GAME (4000 Components) ---

>>> Level-0 Phase...

>>> Level-1 Phase...

>>> Level-2 Phase...

>>> Level-3 Phase...

>>> Level-4 Phase...

>>> Level-5 Phase...

>>> Level-6 Phase...

>>> Level-7 Phase...

>>> Level-8 Phase...

>>> Level-9 Phase...

>>> Level-10 Phase...

>>> Level-11 Phase...

>>> Level-12 Phase...

>>> Level-13 Phase...

>>> Level-14 Phase...

>>> Level-15 Phase...

>>> Level-16 Phase...

>>> Level-17 Phase...

>>> Level-18 Phase...

>>> Level-19 Phase...

>>> Level-20 Phase...

>>> Level-21 Phase...

>>> Level-22 Phase...

>>> Level-23 Phase...

>>> Level-24 Phase...

>>> Level-25 Phase...

>>> Level-26 Phase...

>>> Level-27 Phase...

>>> Level-28 Phase...

>>> Level-29 Phase...

>>> Level-30 Phase...

>>> Level-31 Phase...

>>> Level-32 Phase...

>>> Level-33 Phase...

>>> Level-34 Phase...

>>> Level-35 Phase...

>>> Level-36 Phase...

>>> Level-37 Phase...

>>> Level-38 Phase...

>>> Level-39 Phase...

>>> Level-40 Phase...

>>> Level-